#Actividad 1

In [ ]:
root_path = 'drive/MyDrive/Colab Notebooks/'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import datetime, math
import sys

sys.path.append('/content/' + root_path)

from MRE import Job


Mounted at /content/drive


#1) Implemente un job MapReduce que resuelva la unión de dos datasets (meses).


In [ ]:


inputDir1 = "/content/drive/MyDrive/Colab Notebooks/Actividad1/input/Enero"
inputDir2 =  "/content/drive/MyDrive/Colab Notebooks/Actividad1/input/Febrero"
outputDir = "/content/drive/MyDrive/Colab Notebooks/Actividad1/output"
def fmap(key, value, context):
    context.write(value, 1)            #De esta forma logramos que haya un reducer para cada elemento y como valor enviamos el identificador del conjunto al que pertenece
        
def fred(key, values, context):
    
    context.write(1, key)             #Se escribirá una vez por cada reducer, evitando valores duplicados en el archivo de salida. 

job = Job(inputDir1, outputDir, fmap, fred)
job.addInputPath(inputDir2, fmap)
success = job.waitForCompletion()
print(success)


True


#2)Implemente un job MapReduce que resuelva la intersección de dos datasets (meses)



In [ ]:
inputDir1 = "/content/drive/MyDrive/Colab Notebooks/Actividad1/input/pruebas/primero"
inputDir2 =  "/content/drive/MyDrive/Colab Notebooks/Actividad1/input/pruebas/segundo"
outputDir = "/content/drive/MyDrive/Colab Notebooks/Actividad1/output2"

def fmap(key, value, context):
    context.write(value , 1)               

def fmap2(key, value, context):   #De esta forma logramos que haya un reducer para cada elemento y como valor enviamos el identificador del conjunto al que pertenece
    context.write(value , 2)               


def fred(key, values, context):
    mes1=False
    mes2=False
    for v in values:
      if v == 1 :
        mes1=True
      else:
        mes2=True
    if (mes1 and mes2):
      context.write(key, "")    #Solo se escriben como salida los valores que aparecen en AMBAS entradas (meses)

job = Job(inputDir1, outputDir, fmap, fred)
job.addInputPath(inputDir2, fmap2)
success = job.waitForCompletion()
print(success)

True


#3) Implemente un job MapReduce que permita determinar si dos datasets (meses) son iguales. Dos conjuntos son iguales si ambos tienen los mismos elementos sin importar cuántas veces se repitan dentro de cada conjunto.

In [ ]:
import os
inputDir1 = "/content/drive/MyDrive/Colab Notebooks/Actividad1/input/pruebas/primero"
inputDir2 =  "/content/drive/MyDrive/Colab Notebooks/Actividad1/input/pruebas/segundo"
outputDir = "/content/drive/MyDrive/Colab Notebooks/Actividad1/output3b"

def fmap(key, value, context):
    context.write(value , 1)               

def fmap2(key, value, context):        #De esta forma logramos que haya un reducer para cada elemento y como valor enviamos el identificador del conjunto al que pertenece
    context.write(value , 2)               


def fred(key, values, context):
    mes1=False
    mes2=False
    for v in values:
      if v == 1 :
        mes1=True
      else:
        mes2=True
    if not(mes1 and mes2):            #Si los valores NO aparecen en AMBAS entradas, entonces los conjuntos (meses) son distintos.
      context.write("Distintos", "")

job = Job(inputDir1, outputDir, fmap, fred)
job.addInputPath(inputDir2, fmap2)
success = job.waitForCompletion()
print(success)

#A partir de acá el problema deja de ser big data, el archivo resultante puede ser consultado por su tamaño,
# y si este es mayor a 0b quiere decir que hay distintos y no son iguales
sizeFile= os.stat(outputDir +"/output.txt").st_size
if sizeFile > 0: 
  print("Los conjuntos son distintos.")
else:
  print("Los conjuntos son iguales.")


True
Los conjuntos son distintos.


#4) Indique para cada uno de los ejercicios anteriores si cada uno de los job implementados se pueden generalizar para N conjuntos. Es decir:
#i. Un único job que resuelva la unión de N conjuntos.
      Se podría ya que se generan tantos reducers como claves y
      escriben una vez sola. se deberian configurar con addinputpaths
      las rutas a todos los conjuntos a procesar. 
      El mapper haría lo mismo que ahora, mandaría como clave el valor y como
      valor el identificador del conjunto al que pertenece. Y el reducer 
      escribiría una sola vez para cada clave recibida, a modo de evitar 
      duplicados en la salida.      



#ii. Un único job que resuelva la intersección de N conjuntos.
      
      Se podría siempre y cuando sea un N controlable (meses son 12) hay que
      poner una funcion map para cada conjunto (mes) y  verificar
      que esté en todos los meses. Se consultaría por que aparezca en todos.
      La funcion mapper haría lo mismo, siendo necesario uno para cada conjunto
      y la función reducer verifica la pertenencia del valor a todos los
      conjuntos haciendo 
      if (mes1 and mes2 and ..... and mesN):

#iii. Un único job que permita determinar si N conjuntos son iguales entre si.
#Justifique su respuesta explicando, en caso afirmativo como sería la solución; y en caso negativo, por qué no es posible su resolución con un único Job.

      Al igual que en la intersección, implica tener variables y funciones map
      para cada conjunto (mes) que llega en la clave .
      Es viable siempre y cuando N sea controlable. Al momento de trabajar con 
      un gran número de conjuntos, esta solución podría ser ineficiente ya que
      requiere contemplar la pertenencia del valor a cada uno de estos.
      La función mapper hace lo mismo que en los jobs anteriores, y la función
      reducer consulta por lo opuesto al job anterior: 

                    if not(mes1 and mes2 and mes3 ....and mesN):
                    
      Generando salida solo si el valor no se encuentra en alguno de los
      conjuntos.
      A partir de esto deja de ser big data, el archivo resultante puede ser
      consultado por su tamaño, y si este es mayor a 0b quiere decir que hay
      elementos que no pertenecen a todos los conjuntos , es decir los conjuntos
      no son iguales. 